In [1]:
from pytrends.request import TrendReq
import pandas as pd
from datetime import datetime, timedelta
import time

def get_historical_trends(keyword, start_date, end_date, geo=''):
    """
    Fetch historical Google Trends data for a specific keyword
    
    Args:
        keyword (str): Search term to analyze
        start_date (str): Start date in format 'YYYY-MM-DD'
        end_date (str): End date in format 'YYYY-MM-DD'
        geo (str): Geographic location (default is worldwide)
    
    Returns:
        pandas.DataFrame: Historical trend data
    """
    try:
        # Initialize pytrends
        pytrends = TrendReq(hl='en-US', tz=360)
        
        # Create date range
        start = datetime.strptime(start_date, '%Y-%m-%d')
        end = datetime.strptime(end_date, '%Y-%m-%d')
        
        # Build payload
        kw_list = [keyword]
        pytrends.build_payload(
            kw_list,
            cat=0,
            timeframe=f'{start_date} {end_date}',
            geo=geo
        )
        
        # Get interest over time
        trend_data = pytrends.interest_over_time()
        
        if trend_data.empty:
            print(f"No data found for keyword: {keyword}")
            return None
            
        # Clean up the data
        trend_data = trend_data.drop(columns=['isPartial'])
        trend_data.reset_index(inplace=True)
        trend_data.rename(columns={keyword: 'interest'}, inplace=True)
        
        return trend_data
        
    except Exception as e:
        print(f"Error fetching trends data: {str(e)}")
        return None

def main():
    # Example usage
    keyword = "artificial intelligence"
    start_date = "2023-01-01"
    end_date = "2024-01-01"
    geo = "US"  # Use empty string for worldwide
    
    data = get_historical_trends(keyword, start_date, end_date, geo)
    
    if data is not None:
        print(f"\nTrend data for '{keyword}':")
        print(data.head())
        
        # Optional: Save to CSV
        data.to_csv(f'google_trends_{keyword.replace(" ", "_")}.csv', index=False)
        print(f"\nData saved to google_trends_{keyword.replace(' ', '_')}.csv")

if __name__ == "__main__":
    main()


Trend data for 'artificial intelligence':
        date  interest
0 2023-01-01        37
1 2023-01-08        48
2 2023-01-15        48
3 2023-01-22        56
4 2023-01-29        55

Data saved to google_trends_artificial_intelligence.csv
